In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
from matplotlib import pyplot as plt
import os
import pandas as pd
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem

Define the IDF, IDD and the weather file

In [2]:
idd_file='/usr/local/EnergyPlus-9-0-1/Energy+.idd'
weather_file ='/home/ict4bd/Desktop/Project/Weather_file/ITA_Torino.160590_IWEC.epw'

In [3]:
building = ef.get_building('/home/ict4bd/Desktop/Project/IDF final/on.idf')

In [4]:
insulation = FieldSelector(class_name='Material', object_name='_isolante_01445', field_name='Thickness')
insulationPR = Parameter(selector=insulation, value_descriptor=RangeParameter(0.0001, 0.47),
                         name='Insulation Thickness')

glazing = FieldSelector(class_name='WindowMaterial:Glazing', object_name='38',
                        field_name='Solar Transmittance at Normal Incidence')
glazingPR = Parameter(selector=glazing, value_descriptor=RangeParameter(.7, .94), name='glazing STNI')

window_to_wall = wwr(CategoryParameter(options=[.15,.5,.9]))

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientationPR = Parameter(selector=orientation, value_descriptor=CategoryParameter(options=[180, 270]),
                          name='Orientation')

parameters = [orientationPR , window_to_wall, glazingPR, insulationPR]

/home/ict4bd/py3/lib/python3.6/site-packages/besos/parameters.py:400: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.15, 0.5, 0.9])
  warnings.warn(f'wwr is intended to be used with RangeParameter. Your value_descriptor is {value_descriptor}')


In [5]:
samples_temp =[]
samples_temp.append({'Orientation': 270,
                     'Window to Wall Ratio': 0.15,
                     'glazing STNI': 0.7,
                     'Insulation Thickness': 0.35,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

,Orientation,Window to Wall Ratio,glazing STNI,Insulation Thickness
0,270,0.15,0.7,0.35


In [6]:
objectives = ['Electricity:Facility', 'DistrictHeating:Facility', 'DistrictCooling:Facility']

problem = EPProblem(parameters, objectives)
evaluator = EvaluatorEP(problem, building, out_dir='Simulation_Output/output_besos_best_parameters', error_mode='Silent', err_dir='Simulation_Output/output_besos_best_parameters',
                        epw= weather_file)
outputs = evaluator.df_apply(samples, keep_input=True)    

In [7]:
idf_file = '/home/ict4bd/Desktop/Project/Simulation_Output/output_besos_best_parameters/BESOS_Output/in.idf'
idf = IDF(idf_file,weather_file)

In [8]:
run_period = idf.idfobjects['RunPeriod'][0]
run_period.Begin_Year = 2017
run_period.End_Year = 2019
run_period


RunPeriod,
    BuildLab (01-01:31-12),    !- Name
    1,                        !- Begin Month
    1,                        !- Begin Day of Month
    2017,                     !- Begin Year
    12,                       !- End Month
    31,                       !- End Day of Month
    2019,                     !- End Year
    ,                         !- Day of Week for Start Day
    No,                       !- Use Weather File Holidays and Special Days
    No,                       !- Use Weather File Daylight Saving Period
    Yes,                      !- Apply Weekend Holiday Rule
    Yes,                      !- Use Weather File Rain Indicators
    Yes,                      !- Use Weather File Snow Indicators
    ;                         !- Treat Weather as Actual

In [9]:
idf.run(readvars=True,output_directory=u'Simulation_Output/output_EPPY_3year',annual= True)


/usr/local/EnergyPlus-9-0-1/energyplus --readvars --idd /home/ict4bd/py3/lib/python3.6/site-packages/Data/example_idd.idd --annual --output-directory /home/ict4bd/Desktop/Project/Simulation_Output/output_EPPY_3year --weather /home/ict4bd/Desktop/Project/Weather_file/ITA_Torino.160590_IWEC.epw /home/ict4bd/Desktop/Project/in.idf



In [10]:
csv_columns = ['Date/Time',
              'BLOCCO1:ROOM407:Zone Operative Temperature [C](Hourly)',
              'BLOCCO1:ROOM408:Zone Operative Temperature [C](Hourly)',
              'BLOCCO1:CORRIDOIO:Zone Operative Temperature [C](Hourly)',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
              'DistrictHeating:Facility [J](Hourly)',
              'DistrictCooling:Facility [J](Hourly)',
              'Electricity:Facility [J](Hourly)']
              #'InteriorEquipment:Electricity [J](Hourly)']

In [11]:
df_columns = {'Date/Time':'Date_Time',
              'BLOCCO1:ROOM407:Zone Operative Temperature [C](Hourly)':'t_in_ROOM407',
              'BLOCCO1:ROOM408:Zone Operative Temperature [C](Hourly)':'t_in_ROOM408',
              'BLOCCO1:CORRIDOIO:Zone Operative Temperature [C](Hourly)':'t_in_CORRIDOIO',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)':'t_out',
              'DistrictHeating:Facility [J](Hourly)':'power_heating',
              'DistrictCooling:Facility [J](Hourly)':'power_cooling' ,
              'Electricity:Facility [J](Hourly)':'power_electricity'}
              #'InteriorEquipment:Electricity [J](Hourly)':'power_InteriorEquipment'}

In [12]:
idf_data =pd.read_csv('/home/ict4bd/Desktop/Project/Simulation_Output/output_EPPY_3year/eplusout.csv',usecols=csv_columns)

In [13]:
idf_data = idf_data.rename(columns =df_columns)

In [14]:
idf_data['t_in'] = idf_data[['t_in_ROOM407','t_in_ROOM408','t_in_CORRIDOIO']].mean(axis=1)
idf_data.drop(['t_in_ROOM407', 't_in_ROOM408', 't_in_CORRIDOIO'], axis = 1)

,Date_Time,t_out,power_electricity,power_heating,power_cooling,t_in
0,01/01 01:00:00,0.925,5.019437e+05,0.0,0.0,3.675616
1,01/01 02:00:00,0.650,5.019437e+05,0.0,0.0,3.603681
2,01/01 03:00:00,0.850,5.019437e+05,0.0,0.0,3.601315
3,01/01 04:00:00,0.975,5.019437e+05,0.0,0.0,3.586018
4,01/01 05:00:00,1.000,5.019437e+05,0.0,0.0,3.566741
...,...,...,...,...,...,...
26275,12/31 20:00:00,0.875,9.305593e+06,0.0,0.0,20.799003
26276,12/31 21:00:00,0.575,5.019437e+05,0.0,0.0,19.767256
26277,12/31 22:00:00,0.275,5.019437e+05,0.0,0.0,19.452057
26278,12/31 23:00:00,0.200,5.019437e+05,0.0,0.0,19.113428


In [15]:
idf_data['temp_diff'] =idf_data['t_in'] - idf_data['t_out']

In [16]:
first_year = idf_data[0:8760]
first_year['Date_Time'] = '2017/' + first_year['Date_Time'].str.strip()

In [17]:
second_year = idf_data[8760:17520]
second_year['Date_Time'] = '2018/' + second_year['Date_Time'].str.strip()

In [18]:
third_year = idf_data[17520:]
third_year['Date_Time'] = '2019/' + third_year['Date_Time'].str.strip()

In [19]:
idf_data['Date_Time'] = idf_data['Date_Time'].str.replace('24:00:00','23:59:59')

In [20]:
idf_data['power_heating'] /= 3.6e6
idf_data['power_cooling'] /= 3.6e6
idf_data['power_electricity'] /= 3.6e6

In [21]:
idf_data['power_heating'] = idf_data['power_heating'].apply(lambda x: round(x, 2))
idf_data['power_cooling'] = idf_data['power_cooling'].apply(lambda x: round(x, 2))
idf_data['power_electricity'] = idf_data['power_electricity'].apply(lambda x: round(x, 2))
idf_data['t_in'] = idf_data['t_in'].apply(lambda x: round(x, 2))
idf_data['t_out'] = idf_data['t_out'].apply(lambda x: round(x, 2))

In [22]:
idf_data['total_power'] = idf_data['power_heating']+ idf_data['power_cooling'] #+idf_data['power_electricity']

In [23]:
idf_data = idf_data[['Date_Time','t_in','t_out','temp_diff','power_heating','power_cooling','power_electricity','total_power']]

In [24]:
idf_data.to_csv(path_or_buf='/home/ict4bd/Desktop/Project/Data_Output/simulation_Data_3Year.csv',index=False)